# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_csv = '../WeatherPy/output_data/cities.csv'
city_df = pd.read_csv(city_csv)
city_df.head() # called it weather_df in the weatherpy exercise, but don't want to get confused so let's use a different name

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Datetime
0,Dhārchula,29.85,80.53,60.67,42,0,4.14,IN,1602873510
1,Ushuaia,-54.80,-68.30,46.40,61,75,21.92,AR,1602873493
2,Punta Arenas,-53.15,-70.92,41.00,86,90,11.41,CL,1602873367
3,Albany,42.60,-73.97,50.00,77,100,3.87,US,1602873511
4,Bethel,41.37,-73.41,52.00,93,90,6.93,US,1602873511


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
# locations for setting markers on the map
locations = city_df[['Lat','Lng']]
weights = city_df['Humidity']

fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations,weights=weights,max_intensity=93))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
new_city_df = city_df.loc[(city_df['Max Temp']>70) & (city_df['Max Temp']<80) & (city_df['Wind Speed']<70) 
                         & (city_df['Cloudiness']==0),:]
#there shouldn't be any null values with this approach, but we'll drop them just in case
new_city_df = new_city_df.dropna(how='any')
new_city_df.describe()

,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Datetime
count,23.000000,23.000000,23.000000,23.000000,23.0,23.000000,2.300000e+01
mean,0.903913,36.056522,76.010000,55.347826,0.0,7.053913,1.602874e+09
std,24.584817,64.060258,2.430492,24.350861,0.0,4.750632,4.851376e+01
min,-38.550000,-139.030000,72.680000,18.000000,0.0,1.160000,1.602873e+09
25%,-20.015000,27.395000,73.740000,33.500000,0.0,3.860000,1.602874e+09
50%,-8.770000,40.920000,75.990000,62.000000,0.0,6.350000,1.602874e+09
75%,25.285000,59.700000,78.405000,73.000000,0.0,8.735000,1.602874e+09
max,36.980000,160.700000,79.650000,93.000000,0.0,20.310000,1.602874e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotels_df = new_city_df
hotels_df['Hotel Name'] = ''
hotels_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Datetime,Hotel Name
13,Grand Baie,-20.02,57.58,73.99,73,0,1.99,MU,1602873512,
21,Jiwani,25.05,61.74,79.61,26,0,16.84,PK,1602873513,
50,Atuona,-9.80,-139.03,78.85,73,0,20.31,PF,1602873517,
111,Kiryat Gat,31.61,34.76,75.99,65,0,5.37,IL,1602873526,
116,Yulara,-25.24,130.99,77.00,31,0,8.05,AU,1602873527,


In [8]:
#looks like we don't need to do this multiple times so no need to create a function
hotels = []

params = {'key':g_key,
         'radius':5000,
         'types':'hotel'}

for index,row in hotels_df.iterrows():
    city = row['City']
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f'{lat},{lng}'
    
    url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    print(f'Retrieving results for {city}')
    response_data = requests.get(url, params=params).json()
    results = response_data['results']
    
    try:
        hotels.append(results[0]['name'])
    except:
        print(f'There are no hotels near {city}')

Retrieving results for Grand Baie
Retrieving results for Jiwani
Retrieving results for Atuona
Retrieving results for Kiryat Gat
Retrieving results for Yulara
Retrieving results for Abu Kamal
Retrieving results for Chibombo
Retrieving results for Bandar-e Lengeh
Retrieving results for Necochea
Retrieving results for Grand Gaube
Retrieving results for Kaoma
Retrieving results for Utete
Retrieving results for Salalah
Retrieving results for Chiredzi
Retrieving results for Parlākimidi
Retrieving results for Auki
Retrieving results for San Bernardo
Retrieving results for Tura
Retrieving results for Serowe
Retrieving results for Riyadh
Retrieving results for Dīdwāna
Retrieving results for Piaçabuçu
Retrieving results for Adana Province


In [9]:
hotels; #some of the hotels have the same name as the city so it was confusing..
hotels_df['Hotel Name'] = hotels
hotels_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Datetime,Hotel Name
13,Grand Baie,-20.02,57.58,73.99,73,0,1.99,MU,1602873512,Triolet
21,Jiwani,25.05,61.74,79.61,26,0,16.84,PK,1602873513,Jīwani
50,Atuona,-9.80,-139.03,78.85,73,0,20.31,PF,1602873517,Atuona
111,Kiryat Gat,31.61,34.76,75.99,65,0,5.37,IL,1602873526,Kiryat Gat
116,Yulara,-25.24,130.99,77.00,31,0,8.05,AU,1602873527,Petermann


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
locations = hotels_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))